In [1]:
import sys

sys.path.append('../')

In [2]:
import pickle
from glob import glob
import os
import yaml
from easydict import EasyDict as edict
import numpy as np
import torch

In [3]:
from model.resnet_xor import *
from model.resnet import *

In [4]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, ConcatDataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
test_dataset = datasets.CIFAR10(root='../xor_neuron_data/data',
                                train=False,
                                transform=transform,
                                download=True)

Files already downloaded and verified


In [6]:
test_loader = DataLoader(dataset=test_dataset,
                                 batch_size=16,
                                 shuffle=False)

In [7]:
imgs, target = next(iter(test_loader))

In [8]:
config_file = glob('../config/resnet/xor_resnet.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

control_config_file = glob('../config/control_model/resnet20.yaml')[0]
control_config = edict(yaml.load(open(control_config_file, 'r'), Loader=yaml.FullLoader))

In [9]:
config.model.arg_in_dim = 2
config.model.inner_net = 'quad'

In [10]:
config.model.num_blocks = [3,3,3]
control_config.model.num_blocks = [3,3,3]

In [11]:
model = ResNet20_Xor(config)

In [12]:
model

ResNet_Xor(
  (loss_func): CrossEntropyLoss()
  (inner_net): QuadraticInnerNet(
    (A): Linear(in_features=2, out_features=2, bias=False)
    (b): Linear(in_features=2, out_features=1, bias=True)
  )
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock_InnerNet(
      (inner_net): QuadraticInnerNet(
        (A): Linear(in_features=2, out_features=2, bias=False)
        (b): Linear(in_features=2, out_features=1, bias=True)
      )
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequent

In [13]:
out = model(imgs, target)

In [ ]:
origin_resnet = resnet20(control_config)

In [ ]:
origin_resnet